# Playwright agent

This agent uses Playwright to perform tasks using a web browser. 

In [1]:
# Install the datasets library from huggingface
!pip install opperai -U
!pip install pydantic
!pip install playwright
!playwright install-deps


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: opperai
    Found existing installation: opperai 0.5.3
    Uninstalling opperai-0.5.3:
      Successfully uninstalled opperai-0.5.3
  Using cached greenlet-3.0.3-cp311-cp311-macosx_11_0_universal2.whl.metadata (3.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 26.5 MB/s eta 0:00:0000:0100:01
Using cached greenlet-3.0.3-cp311-cp311-macosx_11_0_universal2.whl (271 kB)


In [3]:
from opperai.types.spans import Span
from playwright.sync_api import sync_playwright
from pprint import pprint
import os

# Opper imports
from opperai import fn, start_span
from pydantic import BaseModel, Field

# Environment configuration
os.environ["OPPER_PROJECT"] = "browser-agent"
os.environ["OPPER_DEFAULT_MODEL"] = "openai/gpt-4o"

# Goal definition
goal = "How do I create an API key on the Opper platform? Docs at https://docs.opper.ai"


# Decision object
class Action(BaseModel):
  observation: str
  thought: str
  plan: str
  next_action: str
  action_input: str = Field(
      None,
      description=
      "The appropriate input to the action. For navigation, it should always be the full URL. For return, it should be the answer."
  )


# Opper function to decide agent action
@fn(path="browser_agent/decide_action", few_shot=True)
def DetermineAction(goal: str, actions: list, trajectory: list) -> Action:
  """ You are a curious and thoughtful agent in persuit of a goal. Use the actions as you see fit and use the trajectory to see what you have been doing before picking the next action. Try not to repeat steps you have already taken, such as visiting the same pages."""


# Help to print outputs
def print_data(data):
  print()
  data = data.dict()
  for key, value in data.items():
    if key in ["thought", "next_action", "action_input"]:
      print(f"{key}: {value}")
  print("-" * 25)


# Run the agent
def run_agent():
  # Potential actions for the agent to take
  actions = ["navigate_and_look", "return"]
  attempts = 15
  trajectory = []

  with sync_playwright() as playwright:
    browser = playwright.chromium.launch(headless=True)
    page = browser.new_page()

    # Trace all Opper calls
    with start_span(name="run", input=goal) as span:

      for _ in range(attempts):

        action = DetermineAction(goal=goal,
                                 actions=actions,
                                 trajectory=trajectory)

        print_data(action)

        if action.next_action == "return":
          print("Result:", action.action_input)
          span.output = action.action_input
          break

        elif action.next_action == "navigate_and_look":
          try:
            page.goto(action.action_input)
            output = {
                "title":
                page.title(),
                "content":
                page.locator('body').inner_text(),
                "url":
                page.url,
                "links": [
                    link.get_attribute("href")
                    for link in page.query_selector_all("a")
                ],
                "forms": [
                    form.get_attribute("action")
                    for form in page.query_selector_all("form")
                ]
            }
          except Exception as e:
            output = {"error": str(e)}

          # Add results to trajectory
          trajectory.append({
              "action": action.next_action,
              "input": action.action_input,
              "output": output
          })

    browser.close()


if __name__ == "__main__":
  run_agent()


ImportError: cannot import name 'async_playwright' from 'playwright.sync_api' (/Users/goransandahl/Development/opper-cookbook/examples/mistral-rag/.venv/lib/python3.11/site-packages/playwright/sync_api/__init__.py)